Taking all data and creating a dataset

In [3]:
#importing stuff
import tensorflow as tf
import pandas as pd
import numpy as np
from tensorflow import keras
import os
from tqdm import tqdm
from matplotlib import pyplot as plt

In [2]:
#read the data
#format: holder = pd.read_csv("filepathname")
folders = ["window 16","window 32","window 5"]
category = ["distance2","abrupt","ideal","obstacle"]
data = []
for i in folders:
    for j in tqdm(category, desc =f"Loading all scenarios of {i}"):
        filenumber = 0
        filepath = os.path.join(i,j,str(filenumber)+".csv")
        while (os.path.isfile(filepath)):
            reading = pd.read_csv(filepath)[['linkQuality','neighborLinkQuality','RSSI value', 'AVG RSSI value','Connected']]

            reading['RSSI value'] = (reading['RSSI value']-reading['RSSI value'].mean())/reading['RSSI value'].std()
            reading['AVG RSSI value'] = (reading['AVG RSSI value']-reading['AVG RSSI value'].mean())/reading['AVG RSSI value'].std()

            data.append(reading)
            filenumber += 1
            filepath = os.path.join(i,j,str(filenumber)+".csv")

test = np.array(data, dtype="float32")
        

Loading all scenarios of window 5: 100%|██████████| 4/4 [00:00<00:00, 16.84it/s]
C:\Users\eyaph\AppData\Local\Temp\ipykernel_15284\4230357887.py:20: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  test = np.array(data)


In [3]:
testdata = pd.read_csv(os.path.join("window 16","distance2","1.csv"))[['linkQuality','neighborLinkQuality','RSSI value', 'AVG RSSI value','Connected']]
column_indices = {name: i for i, name in enumerate(testdata.columns)}
column_indices

{'linkQuality': 0,
 'neighborLinkQuality': 1,
 'RSSI value': 2,
 'AVG RSSI value': 3,
 'Connected': 4}

In [10]:
#modifying the split window function


def split_window(features,input_slice,labels_slice,label_columns,column_indices,input_width,label_width):
    input_columns = ['linkQuality','neighborLinkQuality','RSSI value', 'AVG RSSI value']
    
    inputs = tf.convert_to_tensor(features[input_slice])
    inputs = tf.stack(
            [inputs[ :,column_indices[name]] for name in input_columns],
            axis=-1)
    labels = tf.convert_to_tensor(features[labels_slice])
    if label_columns is not None:
        labels = tf.stack(
            [labels[ :,column_indices[name]] for name in label_columns],
            axis=-1)

    # Slicing doesn't preserve static shape information, so set the shapes
    # manually. This way the `tf.data.Datasets` are easier to inspect.

    inputs.set_shape([input_width, None])
    labels.set_shape( (label_width, None))

    return inputs, labels

In [11]:
def makedataset(features,total_window,input_slice,labels_slice,label_columns,column_indices,input_width,label_width,slide = 1,batch=32):
    

    data = []
    labels = []
    for i in tqdm(features,"Generating dataset"):
        sliding = 0
        i = np.array(i,dtype="float32")
        while(i.shape[0]>(sliding + total_window)):
            input, label= split_window(i[sliding:total_window+sliding],input_slice,labels_slice,label_columns,column_indices,input_width,label_width)
            #we need some collection mechanism here
            data.append(input)
            labels.append(label)
            sliding+=1
    data = tf.convert_to_tensor(data)
    label = tf.convert_to_tensor(labels)
    dataset = tf.data.Dataset.from_tensor_slices((data,label))
    print(dataset)
    test = dataset.batch(batch_size= 32, drop_remainder=True)
    print(dataset.element_spec)
    return test
    
            

input_width = 5
shift = 1
total_window = input_width+shift
label_width = 1
input_slice = slice(0, input_width)
label_start = total_window - label_width
labels_slice = slice(label_start, None)
label_columns = ["Connected"]
finaltest = makedataset(test,total_window,input_slice,labels_slice,label_columns,column_indices,input_width,label_width)
for example_inputs, example_labels in finaltest.take(2):
    print(f'Inputs shape (batch, time, features): {example_inputs.shape}')
    print(f'Labels shape (batch, time, features): {example_labels.shape}')


Generating dataset: 100%|██████████| 70/70 [00:25<00:00,  2.80it/s]


<TensorSliceDataset element_spec=(TensorSpec(shape=(5, 4), dtype=tf.float32, name=None), TensorSpec(shape=(1, 1), dtype=tf.float32, name=None))>
(TensorSpec(shape=(5, 4), dtype=tf.float32, name=None), TensorSpec(shape=(1, 1), dtype=tf.float32, name=None))
Inputs shape (batch, time, features): (32, 5, 4)
Labels shape (batch, time, features): (32, 1, 1)
Inputs shape (batch, time, features): (32, 5, 4)
Labels shape (batch, time, features): (32, 1, 1)


In [6]:
lstm_model = tf.keras.models.Sequential([
    # Shape [batch, time, features] => [batch, time, lstm_units]
    tf.keras.layers.LSTM(5, return_sequences=True),
    # Shape => [batch, time, features]
    tf.keras.layers.Dense(units=1)
])

In [7]:
MAX_EPOCHS = 30

def compile_and_fit(model, window, patience=2):
  early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss',
                                                    patience=patience, verbose = 1,
                                                    mode='min')

  model.compile(loss=tf.keras.losses.MeanSquaredError(),
                optimizer=tf.keras.optimizers.Adam(),
                metrics=[tf.keras.metrics.MeanAbsoluteError()])

  history = model.fit(window, epochs=MAX_EPOCHS, verbose = 1,
                      validation_data=window.take(10),
                      callbacks=[early_stopping])
  return history

In [8]:
history = compile_and_fit(lstm_model, finaltest)



Epoch 1/30
231/231 [==============================] - 19s 12ms/step - loss: 0.7307 - mean_absolute_error: 0.7877 - val_loss: 0.1645 - val_mean_absolute_error: 0.3320
Epoch 2/30
231/231 [==============================] - 2s 8ms/step - loss: 0.0870 - mean_absolute_error: 0.2327 - val_loss: 0.0523 - val_mean_absolute_error: 0.1760
Epoch 3/30
231/231 [==============================] - 2s 7ms/step - loss: 0.0427 - mean_absolute_error: 0.1585 - val_loss: 0.0293 - val_mean_absolute_error: 0.1272
Epoch 4/30
231/231 [==============================] - 1s 6ms/step - loss: 0.0245 - mean_absolute_error: 0.1151 - val_loss: 0.0165 - val_mean_absolute_error: 0.0911
Epoch 5/30
231/231 [==============================] - 2s 8ms/step - loss: 0.0138 - mean_absolute_error: 0.0817 - val_loss: 0.0091 - val_mean_absolute_error: 0.0645
Epoch 6/30
231/231 [==============================] - 2s 7ms/step - loss: 0.0077 - mean_absolute_error: 0.0580 - val_loss: 0.0052 - val_mean_absolute_error: 0.0472
Epoch 7/30
231

In [9]:
testprediction = lstm_model.evaluate(finaltest.take(50))
print(testprediction)

50/50 [==============================] - 0s 4ms/step - loss: 7.6380e-05 - mean_absolute_error: 0.0057
[7.637973612872884e-05, 0.005654463544487953]
